In [21]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

## Neighbourhood wise modelling

In [22]:
def onehot_encode(X, cols):
    # Treat new categories as a new 'unknown' category (all onehot columns are 0)
    onehot_enc = OneHotEncoder(handle_unknown='ignore')
    # Fit encoder on training data
    onehot_enc.fit(X[cols])
    # Get the names of the new columns created
    colnames = list(onehot_enc.get_feature_names(input_features=cols))
    # Transform the data
    onehot_vals = onehot_enc.transform(X[cols]).toarray()
    # Put transformed data into dataframe
    enc_df = pd.DataFrame(onehot_vals,columns=colnames,index=X.index)
    # Add onehot columns back onto original dataframe and drop the original columns
    X = pd.concat([X,enc_df],axis=1).drop(cols,axis=1)
    return X,onehot_enc

In [23]:
## Read the data for modelling
df = pd.read_csv('../data/final_data_for_modelling.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110519 entries, 0 to 110518
Data columns (total 29 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   patientid                                   110519 non-null  float64
 1   appointmentid                               110519 non-null  int64  
 2   gender                                      110519 non-null  object 
 3   scheduledday                                110519 non-null  object 
 4   appointmentday                              110519 non-null  object 
 5   age                                         110519 non-null  int64  
 6   neighbourhood                               110519 non-null  object 
 7   scholarship                                 110519 non-null  int64  
 8   hypertension                                110519 non-null  int64  
 9   diabetes                                    110519 non-null  int64  
 

In [24]:
## get a list of neighbourhoods
neigh_list = list(df.neighbourhood.unique())
filter_neigh_list = [neigh for neigh in neigh_list if len(df[df["neighbourhood"] == neigh])>1000]

filter_neigh_list

['JARDIM DA PENHA',
 'ANDORINHAS',
 'NOVA PALESTINA',
 'DA PENHA',
 'TABUAZEIRO',
 'SÃO PEDRO',
 'SANTA MARTHA',
 'SÃO CRISTÓVÃO',
 'MARUÍPE',
 'GRANDE VITÓRIA',
 'SÃO BENEDITO',
 'ILHA DAS CAIEIRAS',
 'SANTO ANDRÉ',
 'BONFIM',
 'JARDIM CAMBURI',
 'MARIA ORTIZ',
 'JABOUR',
 'RESISTÊNCIA',
 'ILHA DE SANTA MARIA',
 'SANTO ANTÔNIO',
 'BELA VISTA',
 'PRAIA DO SUÁ',
 'ITARARÉ',
 'INHANGUETÁ',
 'SÃO JOSÉ',
 'REDENÇÃO',
 'CENTRO',
 'SANTOS DUMONT',
 'CARATOÍRA',
 'GURIGICA',
 'JOANA D´ARC',
 'CONSOLAÇÃO',
 'PRAIA DO CANTO',
 'FORTE SÃO JOÃO',
 'JESUS DE NAZARETH',
 'SANTA TEREZA',
 'CRUZAMENTO',
 'ILHA DO PRÍNCIPE',
 'ROMÃO']

In [25]:
## Make dataframe filters for each neighbourhood
data = {}
for neigh in filter_neigh_list:
    df_temp = df.copy()
    df_temp = df_temp[df_temp["neighbourhood"] == neigh]
    df_temp.drop(columns=["age_group", "neighbourhood", "patientid", "appointmentid",
                    "scheduledday", "appointmentday", "showed"] , inplace=True)
    data[neigh] = df_temp

In [26]:
## Create one model for each neighbourhood

for key, datafr in data.items():
    # Splitting dataset into test and train
    X_train, X_test, y_train, y_test = train_test_split(datafr.drop(columns=["no_show"]), datafr["no_show"], random_state=0,test_size=0.2)

    ## Ordinal encoder for features
    enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    ## Fit encoder on train and apply to test data as well
    X_train[["gender"]] = enc.fit_transform(X_train[["gender"]])
    X_test[["gender"]] = enc.transform(X_test[["gender"]])

    ## One hot encode the train data
    cols = ["appointment_day_of_week"]
    X_train, onehot_enc = onehot_encode(X_train, cols)

    # Apply onehot encoder to test data
    colnames = columns=list(onehot_enc.get_feature_names(input_features=cols))
    onehot_vals = onehot_enc.transform(X_test[cols]).toarray()

    # Put transformed data into dataframe
    enc_df = pd.DataFrame(onehot_vals,columns=colnames,index=X_test.index)
    # Add onehot columns back onto original dataframe and drop the original columns
    X_test = pd.concat([X_test,enc_df],axis=1).drop(cols, axis=1)

    # Create the parameter grid based on the results of random search 
    param_grid = {
        'max_depth' : [3,4,5,6],
        'min_samples_leaf': [2, 3, 4, 5],
        'n_estimators': [25, 50, 75],
        'random_state':[0],
        'criterion' :['gini', 'entropy'],
        'class_weight': [{1:4}]
    }

    # Create a based model
    rf = RandomForestClassifier()
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                                cv = 3, n_jobs = -1, verbose = 0, scoring='precision')

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    ## Train the best model
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)

    ## Classsification report
    y_pred_train = best_model.predict(X_train)
    print(f"Train Report: {key}")
    print(classification_report(y_train, y_pred_train))
    y_pred_test = best_model.predict(X_test)
    print("Test Report")
    print(classification_report(y_test, y_pred_test))


/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train Report: JARDIM DA PENHA
              precision    recall  f1-score   support

           0       0.92      0.74      0.82      2604
           1       0.33      0.65      0.44       497

    accuracy                           0.73      3101
   macro avg       0.62      0.70      0.63      3101
weighted avg       0.82      0.73      0.76      3101

Test Report
              precision    recall  f1-score   support

           0       0.88      0.70      0.78       642
           1       0.27      0.53      0.36       134

    accuracy                           0.67       776
   macro avg       0.57      0.61      0.57       776
weighted avg       0.77      0.67      0.71       776



/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train Report: ANDORINHAS
              precision    recall  f1-score   support

           0       0.95      0.62      0.75      1396
           1       0.41      0.89      0.56       413

    accuracy                           0.68      1809
   macro avg       0.68      0.75      0.65      1809
weighted avg       0.83      0.68      0.70      1809

Test Report
              precision    recall  f1-score   support

           0       0.88      0.57      0.69       345
           1       0.35      0.75      0.48       108

    accuracy                           0.61       453
   macro avg       0.62      0.66      0.59       453
weighted avg       0.75      0.61      0.64       453



/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


...

Train Report: CRUZAMENTO
              precision    recall  f1-score   support

           0       0.95      0.68      0.79       870
           1       0.43      0.87      0.58       248

    accuracy                           0.72      1118
   macro avg       0.69      0.77      0.68      1118
weighted avg       0.83      0.72      0.74      1118

Test Report
              precision    recall  f1-score   support

           0       0.89      0.63      0.74       224
           1       0.32      0.70      0.44        56

    accuracy                           0.65       280
   macro avg       0.61      0.67      0.59       280
weighted avg       0.78      0.65      0.68       280



/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train Report: ILHA DO PRÍNCIPE
              precision    recall  f1-score   support

           0       0.96      0.58      0.72      1383
           1       0.40      0.92      0.56       429

    accuracy                           0.66      1812
   macro avg       0.68      0.75      0.64      1812
weighted avg       0.83      0.66      0.68      1812

Test Report
              precision    recall  f1-score   support

           0       0.91      0.55      0.69       351
           1       0.35      0.83      0.49       103

    accuracy                           0.61       454
   macro avg       0.63      0.69      0.59       454
weighted avg       0.79      0.61      0.64       454



/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/architkaila/opt/anaconda3/envs/aipi520/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Train Report: ROMÃO
              precision    recall  f1-score   support

           0       0.95      0.58      0.72      1383
           1       0.37      0.89      0.53       388

    accuracy                           0.65      1771
   macro avg       0.66      0.74      0.62      1771
weighted avg       0.82      0.65      0.68      1771

Test Report
              precision    recall  f1-score   support

           0       0.91      0.58      0.71       357
           1       0.30      0.76      0.43        86

    accuracy                           0.62       443
   macro avg       0.61      0.67      0.57       443
weighted avg       0.79      0.62      0.66       443

